In [0]:
dbutils.widgets.text("p_file_date", "")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"


In [0]:
%run "../includes/common_functions"


In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, DateType, TimestampType, FloatType

pitstop_schema = StructType(fields = [
    StructField("raceId", IntegerType(), False),
    StructField("driverId", IntegerType(), True),
    StructField("stop", StringType(), True),
    StructField("lap", IntegerType(), True),
    StructField("time", StringType(), True),
    StructField("duration", StringType(), True),
    StructField("milliseconds", IntegerType(), True),
])

pitstop_df = spark.read.schema(pitstop_schema).option("multiline", True).json(f"{raw_folder_path}/{v_file_date}/pit_stops.json")
display(pitstop_df)

In [0]:
from pyspark.sql.functions import current_timestamp, lit

final_df = pitstop_df.withColumnRenamed("raceId", "race_id") \
                    .withColumnRenamed("driverId", "driver_id") \
                    .withColumn("file_date", lit(v_file_date)) 

final_df = add_ingestion_date(final_df)

display(final_df)

In [0]:
# overwrite_partition(final_df, "f1_processed", "pit_stops", "race_id")

In [0]:
merge_condition = "tgt.race_id = src.race_id AND tgt.driver_id = src.driver_id AND tgt.race_id = src.race_id"
merge_delta_data(final_df, "f1_processed", "pit_stops", processed_folder_path, merge_condition, "race_id")

In [0]:
%sql
SELECT race_id, COUNT(*)
FROM f1_processed.pit_stops GROUP BY race_id
ORDER BY race_id DESC

In [0]:
dbutils.notebook.exit("Success")